# Clean and Combine Player Data

Open necessary functions and set user preferences

In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
import os
import csv

%matplotlib inline

# various options in pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.precision', 3)

# Standard Batting Stats

### Write Standard Batting Stats to DF
Find the standard batting stats table within a player's html page. Return a pandas dataframe containing all major league and minor league seasons for that player.

In [2]:
def get_standard_batting(batter_page):
    batter_standard_data = batter_page.find(
        lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=='batting_standard')

    header_name_list = []
    # Populate a list containing column headers
    header_html = batter_standard_data.find('tr')
    for item in header_html.find_all('th'):
        header_name_list.append(item.text.strip())


    data_html = batter_standard_data.find_all('tr')
    data_list = []
    for ix, row in enumerate(data_html):
        temp_list = []
        if ix >0:
            if 'Yr' in row.find('th').text.strip():
                break
            else:
                #print(row.find('th').text.strip())
                temp_list.append(row.find('th').text.strip())
                for item in row.find_all('td'):
                    temp_list.append(item.text.strip())
                data_list.append(temp_list)

    batting_df = pd.DataFrame(data_list,columns=header_name_list)
    return batting_df

### Clean Batter Stats DF
Convert numeric data from strings to floats.

In [3]:
def clean_batter_df(df):
    numeric_fields = ['Year', 'Age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR',\
                      'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+', \
                      'TB','GDP', 'HBP', 'SH', 'SF', 'IBB']

    text_fields = ['Tm', 'Lg','Pos', 'Awards']
    df_clean = pd.DataFrame()
    
    for nf in numeric_fields:
        df_clean[nf] = pd.to_numeric(df[nf])
    
    for tf in text_fields:
        df_clean[tf] = df[tf]
    
    return df_clean    

## Get Salary Stats
Extra a player's year by year salary data from Baseball reference HTML pages. Return as pandas data frame.

In [4]:
def get_salary_stats(batter_page):
    
    placeholder_list = batter_page.find_all('div', class_='placeholder')

    #div_br-salaries
    for ix, item in enumerate(placeholder_list):
        test_item = item.next_sibling.next_sibling
        test_str = str(test_item)
        if ('div_br-salaries' in test_str) == True:
            salary_table_placeholder = test_item
            
    try: 
        salary_table_placeholder
    except NameError:
        print('No Salary Data')
        return pd.DataFrame()
    else:
        salary_soup = BeautifulSoup(salary_table_placeholder, 'lxml')
        salary_data = salary_soup.find('table')

        header_name_list = []
        # Populate a list containing column headers
        header_html = salary_data.find('tr')
        for item in header_html.find_all('th'):
            header_name_list.append(item.text.strip())


        data_html = salary_data.find_all('tr')
        data_list = []
        for ix, row in enumerate(data_html):
            temp_list = []
            if ix >0:
                year = row.find('th').text.strip()
                if (len(year) <= 4) and ('Yr' not in year):
                    temp_list.append(year)
                    for item in row.find_all('td'):
                        temp_list.append(item.text.strip())
                    while len(temp_list)!=7:
                        temp_list.append('')
                    data_list.append(temp_list)
                else:
                    break

        salary_df = pd.DataFrame(data_list,columns=header_name_list)
        return salary_df

### Clean Batter Salary DF
Clean the copied salary table. Convert numeric data to floats. Replace missing data "?" with numpy nans.

In [5]:
def clean_salary_df(df):
    numeric_fields = ['Year','SrvTm']#['Year', 'Age','SrvTm']
    text_fields = ['Tm', 'Sources','Sources','Notes/Other Sources']
    df = df.replace('?',np.nan)
    
    df_clean = pd.DataFrame()
    
    for nf in numeric_fields:
        df_clean[nf] = pd.to_numeric(df[nf],errors='coerce')
    
    for tf in text_fields:
        if tf == 'Tm':
            df_clean['Tm_full'] = df['Tm']
        else:
            df_clean[tf] = df[tf]
        
    # Convert Salary into numeric value
    df_clean['Salary'] = df['Salary'].replace('[^0-9.]','',regex=True)
    df_clean['Salary'] = pd.to_numeric(df_clean['Salary'])
    
    return df_clean    

### Loop through and make csv files for all players

In [6]:
#### Find Previously Saved Player Files

saved_batter_files = os.listdir('batter_files')
html_files = os.listdir('player_pages')
html_files.remove('.DS_Store')

#### Connect to Baseball Ref Page of Individual Batter

for ix, item in enumerate(html_files):
    html_f = open('player_pages/' + item,'r')
    page = html_f.read()
    batter_page = BeautifulSoup(page,"lxml")
    html_f.close()

    info_header = batter_page.find('p')
    player_id = item.replace('.shtml','')
    
    if 'pitcher' not in str.lower(info_header.text):
        
        # Batting Stats
        batting_df = get_standard_batting(batter_page)
        batting_df_clean = clean_batter_df(batting_df)
        batting_df_clean['player_id'] = player_id
        batting_df_clean
        
        # Salary Stats
        salary_df = get_salary_stats(batter_page)
        if salary_df.empty != True:
            salary_df_clean = clean_salary_df(salary_df)
            salary_df_clean['player_id'] = player_id
            combo_df = pd.merge(batting_df_clean, 
                                salary_df_clean, 
                                how = 'left',
                                on = ['Year','player_id'])

            try:
                combo_df = combo_df[np.isfinite(combo_df['Year'])]
            except TypeError:
                print('Warning: Bad year in combo_df')

            fname = 'batter_files/'+player_id+'.csv'
            combo_df.to_csv(fname,mode = 'w', index=False)
        print(player_id)


abadan01
abbotje01
abbotku01
abercre01
abernbr01
abnersh01
abreubo01
abreujo02
abreuto01
ackledu01
adamecr01
No Salary Data
adamsda02
No Salary Data
adamsla01
adamsma01
adamsru01
No Salary Data
adamsry01
adducji02
adriaeh01
afenitr01
agbaybe01
aguilch01
aguilje01
ahmedni01
alberha01
alcanar01
alcanis01
aldremi01
aldrico01
alexama02
No Salary Data
alfarja01
No Salary Data
alfarjo01
alfoned01
alfonel01
alicelu01
allanan01
allenbr01
allench01
allendu01
allenje01
No Salary Data
allenlu01
allrebe01
almonab01
almoner01
almonzo01
almoral01
alomaro01
alomasa02
alonsyo01
aloumo01
altheaa01
altuvjo01
No Salary Data
alvarcl01
No Salary Data
alvarda02
alvarga01
alvarpe01
No Salary Data
alvarto01
amarari01
amarial01
amaroru02
No Salary Data
ambrech01
amezaal01
anderbr01
anderbr03
No Salary Data
anderbr05
anderda02
No Salary Data
anderdr01
anderga01
anderjo03
anderke01
anderla03
anderma02
anderti01
andinro01
andresh01
andruel01
No Salary Data
anglema01
anthoer01
antonma01
aokino01
arciaor01
arciaos0

bottsja01
bourgja01
bourjpe01
bourju01
bournmi01
bournra01
bowenro01
No Salary Data
bowerbr01
No Salary Data
bowieji01
bowkejo01
bradlja02
bradlmi01
bradlph01
bradlsc01
bradste01
No Salary Data
bradydo01
braggda01
bragggl01
bransje01
brantmi02
brantro01
branyru01
braunry02
No Salary Data
brazecr01
breamsi01
bredebr01
bregmal01
brentbr01
brettge01
No Salary Data
brettry01
brewero01
brignre01
brilegr01
No Salary Data
britobe01
britojo01
No Salary Data
britoju01
britoso01
britoti01
brockgr01
brockta01
brognri01
brookhu01
No Salary Data
brookje01
brookto01
brosisc01
brousbe01
brownad01
brownan02
brownbr01
No Salary Data
brownco01
brownde02
browndo01
brownem01
No Salary Data
brownga02
brownja03
brownje01
No Salary Data
brownje02
No Salary Data
brownjo05
brownke04
brownma03
No Salary Data
brownma05
brownro02
browntr01
broxtke01
bruceja01
bruetj.01
No Salary Data
brumbcl01
brumfja01
brumlmi02
brunato01
brunter01
bryankr01
No Salary Data
bubelja01
buchabr01
buckjo01
bucknbi01
bucktr01
buddery0

No Salary Data
colbecr01
colbrgr01
coleal01
No Salary Data
colemdu01
colemmi01
colemvi01
colesda01
colest01
No Salary Data
colinal01
collida02
collilo01
collity01
colonch01
No Salary Data
coloncr01
colvity01
confomi01
congeha01
coninje01
conrabr01
constjo01
contija01
contrwi01
cooksbr01
No Salary Data
coolbmi01
coolbsc01
coomero01
No Salary Data
coopeda01
No Salary Data
coopega02
coopesc01
coquitr01
coraal01
corajo01
cordewi01
cordoma01
corpoca01
correca01
No Salary Data
correro01
No Salary Data
cortefe01
No Salary Data
costami01
costash01
costech01
costoti01
cotahu01
cottohe01
counscr01
cousisc01
No Salary Data
cowarka01
cowgico01
coxda02
coxst01
cozarza01
crabbca01
No Salary Data
cradlri01
craigal01
crawfbr01
crawfca02
credejo01
crespce01
crespfe01
crispco01
No Salary Data
cromawa01
cromed.01
crometr01
No Salary Data
cronch01
croncj01
crosbbo01
crosbbu01
crowetr01
No Salary Data
crozier01
cruzde01
cruzen01
No Salary Data
cruzfa01
cruziv01
cruzja01
cruzjo02
cruzlu01
cruzne02
cruzto03


No Salary Data
farrier01
fasansa01
No Salary Data
feathta01
febleca01
federti01
feldemi01
No Salary Data
felicje01
felixju01
felizpe01
fermife01
No Salary Data
fernajo01
fernato01
fickro01
fieldce01
No Salary Data
fieldda01
fieldjo02
fieldpr01
No Salary Data
fieldth01
figgami01
figgich01
No Salary Data
figuebi01
No Salary Data
figueco01
No Salary Data
figuelu01
finlest01
No Salary Data
fioreje01
fiskca01
fitzgmi02
flahejo01
flahery01
fletcda01
fletcsc01
florake01
floreje02
No Salary Data
florejo01
florera02
florewi01
floripe01
flowety01
floydcl01
foleyto02
fontemi01
fonvich01
No Salary Data
forbep.01
fordcu01
No Salary Data
fordda03
fordle01
fordybr01
forsylo01
fowlede01
foxan01
foxer01
foxja02
francbe01
francje02
francju01
francju02
francma01
francma02
francte01
frandke01
No Salary Data
frankmi01
No Salary Data
frankmi02
frankni01
fraziad01
No Salary Data
frazije01
frazilo01
frazito01
freelry01
freemch01
freemfr01
No Salary Data
freemmi01
freesda01
freimna01
No Salary Data
freiral01
f

haltesh01
No Salary Data
haltose01
hamelbo01
hamilbi02
hamilda02
hamilje01
hamiljo03
No Salary Data
hamilma01
hammoje01
hammoro01
haneyto01
hanigmi01
hanigry01
hannaja01
hanseda01
hanseje01
hansoal01
No Salary Data
hardtja01
hardyjj01
No Salary Data
haresh01
No Salary Data
harmabr01
harpebr01
No Salary Data
harpebr02
harpebr03
harribr01
No Salary Data
harrido01
harrijo05
harrile01
harriwi02
hartbo01
hartco01
No Salary Data
hartja01
harveke01
haselbi01
No Salary Data
hassaal01
hassero01
hatchbi01
No Salary Data
hatchch01
hatchmi01
hattesc01
No Salary Data
hattijo01
hawpebr01
hayesbr01
hayesch01
hayesvo01
haynena01
hazelje01
No Salary Data
headje01
headlch01
healyry01
heathmi02
No Salary Data
heathsl01
hechaad01
hedgeau01
heepda01
No Salary Data
heffebe01
heintch01
heisech01
helfaer01
helmswe01
heltoto01
hemonsc01
No Salary Data
hemphbr01
hendeda01
henderi01
henlebo01
No Salary Data
hensodr01
heredgu01
hermach01
hermije01
hernaal01
hernaan01
hernaca01
hernaca02
hernace01
hernace02
hernad

...

martira03
martiru01
martisa01
martiti02
martivi01
marzajo01
mashoda01
masteda01
mastrda01
mateohe01
mateoru01
mathejo02
mathemi01
mathije01
No Salary Data
matosfr01
No Salary Data
matosju01
matoslu01
No Salary Data
matospa01
No Salary Data
matrada01
matsuhi01
matsuka01
matthga02
mattido01
No Salary Data
mattike01
mauerjo01
maurero01
No Salary Data
maxwebr01
maxweja01
maxweju01
maybejo02
maybica01
mayde01
No Salary Data
maylu01
maynebr01
maysoed01
No Salary Data
mazalu01
mazarno01
mcanupa01
No Salary Data
mcbrima02
mccanbr01
mccanja02
mccarda01
No Salary Data
mcclasc01
mcclell01
mccoymi01
mccraqu01
No Salary Data
mccraro01
mccutan01
No Salary Data
mcdante01
No Salary Data
mcdavra01
mcdonda02
No Salary Data
mcdondo01
mcdonja02
mcdonjo03
No Salary Data
mcdonke01
mcdouma01
mcdowod01
mcewijo01
mcgeewi01
mcgehca01
No Salary Data
mcginru01
mcgrifr01
mcgrite01
No Salary Data
mcguich01
mcguiry01
mcgwima01
mcintti01
mckayco01
No Salary Data
mckeewa01
mckenmi01
mcknije01
mclemma01
mclouna01
mcmil

odorro01
No Salary Data
oeltjtr01
oestero01
offerjo01
ojedaau01
ojedami01
No Salary Data
olandji01
olerujo01
olivajo01
olivehe01
olivejo01
olivomi01
olmedra01
olsongr02
No Salary Data
olsonma02
No Salary Data
olsonti01
oltmi01
omallsh01
oquenjo01
ordazlu01
ordonma01
ordonre01
orieke01
orlanpa01
orrpe01
orsuljo01
No Salary Data
ortegbi01
No Salary Data
ortegra01
ortizda01
ortizhe01
ortizja01
ortizjo04
ortizju01
ortizlu01
ortmeda01
ortonjo01
osikke01
otanewi01
oterori01
overbly01
owenser01
owensja02
owensje01
owensp01
owingch01
ozunama01
ozunapa01
pachejo01
No Salary Data
padiljo01
paganan01
paglimi01
No Salary Data
pagnoma01
pagnoto01
palacre01
palmede01
palmeor01
palmera01
panikjo01
pankoji01
pappaer01
paquecr01
paredji01
paredjo01
parenma01
parkby01
parkeda01
parkeja03
No Salary Data
parkeky01
parkeri01
parksde01
parmech01
parrage01
parrian01
parrila02
No Salary Data
pascuva01
pasquda01
pastoty01
patteco01
patteer01
No Salary Data
patteja01
pattejo01
No Salary Data
pattejo03
pauliro01

No Salary Data
reeseke01
reesepo01
No Salary Data
refsnro01
reimeke01
reimono01
relafde01
No Salary Data
rendato01
rendoan01
renfrhu01
renteed01
renteri01
repkoja01
restomi01
reverbe01
No Salary Data
reyesar01
reyesgi01
reyesjo01
No Salary Data
reyesjo02
reyesre01
reynoha01
reynoma01
No Salary Data
reynoma03
reynor.01
reynoro02
rhodeka01
rhymewi01
richaan01
richach01
richada02
richaje01
No Salary Data
richake02
rickajo01
No Salary Data
riesgni01
riggash01
riggsad01
rileser01
riosal01
riosar01
ripkebi01
ripkeca01
rivaslu01
No Salary Data
riverca01
No Salary Data
riverca02
riverju01
riverlu01
rivermi02
riverre01
riverru01
rivertj01
riverya01
rizzoan01
roberbi01
roberbr01
No Salary Data
roberch02
roberda07
roberda09
roberke01
robermi01
roberry01
No Salary Data
robidbi01
No Salary Data
robinch02
robincl01
No Salary Data
robinde02
robinke02
robinsh01
No Salary Data
robintr01
No Salary Data
robleos01
rodrial01
rodrica01
No Salary Data
rodried04
No Salary Data
rodrigu01
No Salary Data
rodrigu

smithbo06
smithdw01
smithgr01
smithja05
No Salary Data
smithke04
smithlo01
smithma03
smithma05
smithoz01
smithse01
No Salary Data
smithst02
smoakju01
smolija01
No Salary Data
sneades01
snellch02
snidetr01
snopech01
snowj.01
No Salary Data
snydebr02
snydebr03
snydech02
snydeco02
No Salary Data
snydeea01
sogarer01
sojolu01
solando01
No Salary Data
solanjh01
solarya01
solerjo01
No Salary Data
solisal01
No Salary Data
sorenza01
soriaal01
sorrepa01
No Salary Data
sosaju01
sosasa01
sotoge01
sotone01
souzast01
spande01
spangco01
No Salary Data
spearna01
spehrti01
spencsh01
spierbi01
spiezsc01
spilbry01
spiveju01
spraged02
springe01
No Salary Data
sprinst01
stahosc01
stairma01
No Salary Data
stallja01
stankan01
No Salary Data
stanlco01
stanlmi02
stanscr01
stantmi03
starkma01
stassma01
statoda01
stavini01
steinte01
No Salary Data
stensde01
stephph01
stephra01
sternad01
stevele01
steveto01
No Salary Data
stewaan01
stewach01
stewaia01
stewash01
stillku01
stinnke01
stockke01
No Salary Data
stockme

vittejo01
vizcajo01
vizquom01
No Salary Data
vogelda01
vogtst01
voigtja01
vottojo01
No Salary Data
wakamdo01
No Salary Data
wakelch01
walbema01
No Salary Data
waldrky02
No Salary Data
walewji01
walkech01
No Salary Data
walkech02
walkegr01
walkela01
walkene01
walketo04
wallabr01
wallati01
wallide01
walshco02
walteda01
walteza01
waltoje01
wardda01
wardga01
wardke01
wardtu01
washicl01
washiri01
No Salary Data
waszgb.01
No Salary Data
wathadu01
No Salary Data
watkilo01
watkipa01
No Salary Data
watkito01
watsobr01
No Salary Data
watsoma02
webstle01
webstmi01
wedgeer01
weeksje01
weeksri01
wehnejo01
weisswa01
wellsca01
wellsve01
wendljo01
werthja01
No Salary Data
wessoba01
wheelry01
No Salary Data
wheelze01
whitalo01
whitede03
whitede04
whiteel03
whitefr01
whitejo03
whitema01
whitero02
whitety01
whitmda02
whitter01
widgech01
wietema01
wiggity01
wilkebr01
wilkecu01
No Salary Data
wilkian01
wilkiri01
willaje01
No Salary Data
willian01
willibe02
willied01
willige02
willige03
No Salary Data
willi

# Combine all player files into a single csv

In [7]:
saved_batter_files = os.listdir('batter_files/')

all_batters_file = open('all_batters.csv', mode = 'w+')
all_batters_writer = csv.writer(all_batters_file, delimiter=',')

for ix, bf in enumerate(saved_batter_files):
    if '.csv' in bf:
        batter_data = open('batter_files/' + bf,'r')
        b_file = csv.reader(batter_data,delimiter=',')
        
        if ix == 0:
            for jx, row in enumerate(b_file):
                all_batters_writer.writerow(row)
        else:
            for jx, row in enumerate(b_file):
                if jx > 0:
                    all_batters_writer.writerow(row)
        batter_data.close()
all_batters_file.close()

'zychto01'

### Test to make sure the file was written successfully. 

In [10]:
big_DF = pd.read_csv('all_batters.csv')

In [11]:
len(big_DF['player_id'].unique())

2366